# ***<p style="text-align:center;">Análise de Texto através de Bases de Dados Vetoriais</p>***

Realizado pelo Grupo 14, constituído por:
- José Loureiro, A96467
- José Ferreira, A96798
- Pedro Gonçalves, A101250

## **Resumo**

Este relatório representa o processo de desenvolvimento e implementação de uma Base de Dados Vetorial com o objetivo de serem identificados padrões de texto.

Como Base de Dados vetorial decidimos utilizar **ChromaDB** devido à sua excelente integração com Python aliada com a sua facilidade de configuração.  
O Projeto dividiu-se essencialmente em 3 fases. **Aquição e Processamento de Dados**, **Geração de Embeddings e Armazenamento Vetorial** e **Pesquisa por Similaridade**. Em seguida abordamos cada uma detalhadamente.

Atualmente, é possível carregar variados tipos de ficheiros e pesquisar por um termo ou texto sendo obtido os ficheiros com o conteúdo mais similar.

## **Introdução**

Como mencionamos anteriormente o nosso projeto foi essencialmente dívido em 3 fases.

Relativamente à Aquisição e Processamendo de Dados, a nossa aplicação está preparada para ler ficheiros de texto de 3 tipos: PDF, TXT e DOCX. O conteúdo destes ficheiros é lido e guardado num dicionário onde a chave é o *path* do ficheiro e o valor o conteúdo do mesmo.

Quanto à Geração de Embeddings e Armazenamento Vetorial, usamos o modelo default do ChromaDB *"all-MiniLM-L6-v2"* que irá converter o texto para um vetor e armazená-lo na Base de Dados.

Por fim, é efetuada a procura por similaridade. O Modelo converte a query (input do utilizador) e compara com os vetores previamente armazenados. Neste caso, o modelo escolhido pelo grupo usa a Distância Euclidiana para calcular, onde a proximidade entre vetores representa a similaridade semântica. De seguida, retorna o número de resultados requisitado.

Importação biblioteca para usar o sistemas de base de dados Chromadb

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

Importação de funcionalidades cridas fazer a leitura de arquivos a serem usados durante este projeto

In [ ]:
import leitor
import os

Criar o cliente para o ChromaDB

In [ ]:
chroma_client = chromadb.Client(
    chromadb.config.Settings(chroma_server_host="chroma", chroma_server_http_port="8000")
)

Relativamente ao modelo de embedding escolhido, com base no benchmark do ChromaDB, o modelo *"all-MiniLM-L6-v2"* é um dos melhores modelos disponíveis para a tarefa de embeddings de texto. Para além disso, com base na imagem abaixo, podemos ver que o modelo *"all-MiniLM-L6-v2"* é um dos modelos mais equilibrados, tendo um tamanho reduzido e uma boa performance em termos de velocidade de embedding e de pesquisa.

![alt text](../images/benchmark.jpg)

Para complementar estes benchmarks, realizamos alguns testes de performance com diferentes modelos de embeddings, testando a velocidade de pesquisa. O teste consistiu em realizar uma pesquisa por um termo específico e medir o tempo necessário para retornar os resultados. Utilizamos um conjunto de documentos variados para garantir que os testes fossem representativos.
 
Os resultados foram os seguintes:

![all-MiniLM-L6-v2](../images/all-MiniLM-L6-v2.jpg)![alt text](../images/all-mpnet-base-v2.jpg)![alt text](../images/multi-qa-mpnet-base-dot-v1.jpg)![alt text](../images/paraphare-albert-small-v2.jpg)![alt text](../images/paraphrase-multilingual-mpnet-base-v2.jpg)

Com base nos resultados obtidos, optamos por utilizar o modelo *"all-MiniLM-L6-v2"*

In [ ]:
embedder = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

Neste ponto podemos criar a coleção de embeddings numa estrutura de dados predefinida do ChromaDB.

No ChromaDB uma coleção é o local fundamental onde se armazenam e organizam os embeddings vetoriais e os dados originais que estes representam, no caso do nosso projeto trabalha com dados do tipo .docx, .pdf e .txt.

Assim sendo, na criação da coleção, é necessário definir o nome da coleção  ( name="my_collection" ) e o modelo de embeddings ( embedding_function=embedder, sendo a variável embedder definida pelo Utilizador anteriormente) que irá efetuar a conversão de texto para vetor.

In [ ]:
collection = chroma_client.get_or_create_collection(
        name="my_collection",
        embedding_function=embedder, 
)

Após a criação da coleção, criamos um dicionário vazio para armazenar os textos dos ficheiros lidos, onde o *path* do ficheiro é a chave e o conteúdo do ficheiro é seu o valor.

In [ ]:
dados = {}

Função criada com o objetivo de adicionar o conteudo do nosso dicionário `dados` à coleção.

In [ ]:
def adiciona_dados(collection, dados):
    # Adicionar ficheiros à coleção
    collection.add(
        documents=list(dados.values()),  # lista de ficheiros
        ids=[k for k in dados.keys()]    # ids dos ficheiros
    )
    print(f"Foram adicionados {len(dados)} ficheiros.")
    # Limpar o dicionário para evitar adicionar os mesmos ficheiros novamente
    dados.clear()

No ficheiro Leitor.py, presente no diretório da app, é criada a função `ler_ficheiros` que recebe o caminho do diretório onde se encontram os ficheiros a serem lidos. Esta função percorre todos os ficheiros do diretório e lê o conteúdo de cada um, armazenando-o no dicionário `dados`. 

Ler ficheiros pdf e armazenar o conteúdo na coleção

In [ ]:
leitor.extract_text(r"..\data/pdf", dados)
adiciona_dados(collection, dados)

Ler ficheiros docx e armazenar o conteúdo na coleção

In [ ]:
leitor.extract_text(r"..\data/txt", dados)
adiciona_dados(collection, dados)

Ler ficheiros txt e armazenar o conteúdo na coleção

In [ ]:
leitor.extract_text(r"..\data/docx", dados)
adiciona_dados(collection, dados)

Para efetuar a pesquisa por similaridade, o utilizador escreve o termo ou texto que deseja pesquisar e o número de resultados que deseja obter. Através da função query o termo ou texto é convertido para um embedding e comparado com os embeddings armazenados na coleção, armazenando os resultados mais semelhantes na variável "results".

In [ ]:
prompt = input("Digite o termo que deseja pesquisar: ")
n_resultados = int(input("Quantos resultados deseja obter:  "))
results = collection.query(
        query_texts=[prompt],    # o que o utilizador quer pesquisar                  
        n_results=n_resultados   # Número de resultados a serem retornados
)

Após serem obtidos os resultados da pesquisa, processamos os resultados obtidos, extraindo os textos dos ficheiros correspondentes aos embeddings mais semelhantes encontrados na coleção, assim como o nome do ficheiro e a distância Euclideana entre o embedding de consulta e os embeddings encontrados na coleção. A distância Euclidiana é uma medida de similaridade entre vetores, assumindo um valor no intervalo $[0,+∞[$ sendo que quanto mais proximo o valor de 0, mais similares são os vetores.

In [ ]:
num_results = len(results['documents'][0])
doc_text = []
titles = []
distance = []
for i in range(num_results):
    doc_text.append(results['documents'][0][i])
    distance.append(results['distances'][0][i])
    doc_id_path = results['ids'][0][i]
    titles.append(os.path.basename(doc_id_path))


Apresentamos ao utilizador os títulos dos ficheiros correspondentes aos embeddings mais semelhantes encontrados na coleção, juntamente com a distância entre o embedding de consulta e os embeddings encontrados na coleção.

In [ ]:
print(f"\n--- Pesquisas encontradas ---\n")   
j = 0 
for title in titles:
    print(f"--- Resultado {j+1} ---")
    print(f"Ficheiro: {title}")
    print(f"Distância de Similaridade: {distance[j]:.4f}\n")
    j += 1


O utilizador pode então selecionar o resultado que deseja visualizar, sendo exibido o conteúdo do ficheiro. 

In [ ]:

choice = input("Digite o número do resultado desejado: ")
try:
    choice = int(choice) - 1  # Ajustar para índice 0
    if 0 <= choice < num_results:
        print("============================================\n")
        print(f"Conteúdo do Resultado {choice + 1}:")
        print(f"Ficheiro: {titles[choice]}")
        print(f"{doc_text[choice]}\n\n")
        
    else:
        print("Opção inválida!")
except ValueError:
    print("Por favor, escreva apenas números.")



Para finalizar, também é possível eliminar dados da coleção, é apresentada ao utilizador uma lista com os ids (*path* do ficheiro) dos embeddings que pertencem à coleção. O utilizador pode escolher o id do embedding que deseja eliminar, sendo este removido da coleção.

In [ ]:
ids = collection.get(include=[])
for id in ids['ids']:
    print(id)
id = input("Digite o ID do ficheiro que deseja eliminar: ")
if id in ids['ids']:
    collection.delete(ids=[id])
    print(f"Ficheiro {id} eliminado com sucesso.")
else:
    print(f"Ficheiro {id} não encontrado.")

## **Conclusão**

Neste projeto, desenvolvemos competências em Python, especialmente na manipulação de ficheiros e na utilização de Bases de Dados Vetoriais. A partir do ChromaDB conseguimos explorar e consolidar conhecimentos sobre Bases de Dados Vetoriais, bem como a sua integração com modelos de Machine Learning para a geração de embeddings.

Relativamente a possíveis melhorias, podemos considerar a implementação de uma interface gráfica para facilitar a interação do utilizador com a aplicação, bem como ao invés de ser guardado o conteúdo inteiro do ficheiro num só vetor, poderíamos guardar *`chunks`* de texto, ou seja, pedaços de texto menores, o que poderia melhorar a precisão da pesquisa por similaridade aliado com a implementação de *`LLMs`* (*Large Language Models*) para melhorar a resposta da aplicação às consultas do utilizador, tornando-a mais inteligente e adaptativa.
## **Referências**
https://www.trychroma.com/

https://www.couchbase.com/blog/embedding-models/#:~:text=Embedding%20models%20are%20a%20type,%2C%20low%2Ddimensional%20vector%20space

https://medium.com/@nay1228/embedding-models-a-comprehensive-guide-for-beginners-to-experts-0cfc11d449f1

https://www.sbert.net/docs/sentence_transformer/pretrained_models.html